# Reproducing and Simulating Literature Results Using Pyglotaran: A Step-by-Step Guide

The goal of this notebook is to illustrate how one might be able to re-create an analysis results from the (published) literature using pyglotaran, a steady hand and some basic Python skills.

To illustrate the process, we'll (randomly†) select an example paper from the literature.

<sub>†Random in the sense that the author of the notebook randomly recieved a question about it.</sub>

> **Triplet Formation by Charge Recombination in Thin Film Blends of Perylene Red and Pyrene: Developing a Target Model for the Photophysics of Organic Photovoltaic Materials**  
> *René M. Williams, Nguyễn Vân Anh, and Ivo H. M. van Stokkum*  
> **The Journal of Physical Chemistry B**, 2013, 117 (38), 11239-11248  
> [DOI: 10.1021/jp402086p](https://doi.org/10.1021/jp402086p)

<details>
<summary>Click to show abstract figure (hide on publish)</summary>
<img src="https://pubs.acs.org/cms/10.1021/jp402086p/asset/images/medium/jp-2013-02086p_0010.gif" alt="Abstract Figure" width="400"/>
<!-- This is a link to the original source to avoid copyright issues. -->
<!-- When publishing the notebook make sure the figure does *not* show. -->
</details>



We will be looking at a variant of the target model depicted in the abstract of the paper, specifically the one found in the (freely available) [Supporting Information](https://pubs.acs.org/doi/suppl/10.1021/jp402086p/suppl_file/jp402086p_si_001.pdf) (SI) *Figure S8*, together with the time traces and spectra in *Figure S9*.

Our goal will be able to approximately reproduce the results found in the paper, with *just* the information in the (SI of the) paper.

For our convenience (and to avoid copyright issues) we have re-created the target scheme depicted in Figure S8 using draw.io (see `figures` folder for the editable source).

![img](figures/Figure_S8_Target_Scheme-dark.drawio.svg)

Later we will interpret this scheme as a pyglotaran kinetic scheme and use it, together with the species' spectral shapes, to (approximately) simulate the original data.

## Helper functions

Some helper functions will be needed to clean up and interpolate digitized data and turn it into `xarray` `DataArray`'s.

These functions work with `list` because that's easily obtained (copy pasted) from a digitizer, or written down.

<sub>For now these helper functions are part of this notebook, but it is not impossible they will be absorbed into the `pyglotaran-extras` package at some point.


In [ ]:
import numpy as np
import xarray as xr
from scipy.interpolate import interp1d


def remove_duplicates(x: list[float], y: list[float]) -> tuple[list[float], list[float]]:
    """
    Removes duplicate x-values and their corresponding y-values.

    Parameters
    ----------
    x : list of float
        The x-values.
    y : list of float
        The y-values.

    Returns
    -------
    tuple of list of float
        The x and y values with duplicates removed.
    """
    seen = {}
    duplicates = set()

    # Identify duplicates
    for i, value in enumerate(x):
        if value in seen:
            duplicates.add(i)
        else:
            seen[value] = i

    # Remove duplicates
    x_cleaned = [value for i, value in enumerate(x) if i not in duplicates]
    y_cleaned = [value for i, value in enumerate(y) if i not in duplicates]

    # Print the number of duplicates removed
    num_duplicates = len(duplicates)
    print(f"Number of duplicates removed: {num_duplicates}")

    # If fewer than 5 duplicates, print the exact duplicates removed
    if num_duplicates < 5:
        duplicates_removed = [(x[i], y[i]) for i in duplicates]
        print("Duplicates removed:", duplicates_removed)

    return x_cleaned, y_cleaned


def interpolate_with_scipy(
    x: list[float], y: list[float], num_points: int = 200
) -> tuple[np.ndarray, np.ndarray]:
    """
    Interpolates the given x and y data using scipy's interp1d.

    Parameters
    ----------
    x : list of float
        The x-values.
    y : list of float
        The y-values.
    num_points : int, optional
        The number of points for interpolation. Default is 200.

    Returns
    -------
    tuple of np.ndarray
        The interpolated x and y values.
    """
    interp_func = interp1d(x, y, kind="linear")
    x_new = np.linspace(np.min(x), np.max(x), num_points)
    y_new = interp_func(x_new)
    return x_new, y_new


def interpolate_with_xarray(
    x: list[float],
    y: list[float],
    xrange: tuple[float, float] | None = None,
    num_points: int = 200,
) -> tuple[np.ndarray, np.ndarray]:
    """
    Interpolates the given x and y data using xarray's interpolation.

    Parameters
    ----------
    x : list of float
        The x-values.
    y : list of float
        The y-values.
    xrange : tuple of float, optional
        The range of x-values for interpolation, given as (min, max).
        If None, the range of the input x-values is used. Default is None.
    num_points : int, optional
        The number of points for interpolation. Default is 200.

    Returns
    -------
    tuple of np.ndarray
        The interpolated x and y values.
    """
    if xrange is None:
        xrange = (np.min(x), np.max(x))
    ds = xr.Dataset({"y": ("x", y)}, coords={"x": x})
    x_new = np.linspace(xrange[0], xrange[1], num_points)
    y_new = ds.interp(x=x_new).y.values
    return x_new, y_new


def convert_to_xarray(x: list[float], y: list[float], xlabel: str = "spectral") -> xr.DataArray:
    """
    Converts the given x and y data to an xarray DataArray.

    Parameters
    ----------
    x : list of float
        The x-values.
    y : list of float
        The y-values.
    xlabel : str, optional
        The label for the x-axis. Default is "spectral".

    Returns
    -------
    xr.DataArray
        The xarray DataArray.
    """
    da = xr.DataArray(y, coords={xlabel: x}, dims=[xlabel])
    return da


def linear_log_sampling(
    lin_min: float, lin_max: float, log_max: float, n_points_lin: int, n_points_log: int
) -> np.typing.NDArray[np.float64]:
    """
    Create a linear-logarithmic sampling.

    Parameters
    ----------
    lin_min : float
        Minimum value for linear sampling.
    lin_max : float
        Maximum value for linear sampling and minimum for logarithmic sampling.
    log_max : float
        Maximum value for logarithmic sampling.
    n_points_lin : int
        Number of points to sample linearly.
    n_points_log : int
        Number of points to sample logarithmically.

    Returns
    -------
    npt.NDArray[np.float64]
        Array of sampled points.

    """
    # Linear sampling from lin_min to lin_max
    linear_samples = np.linspace(lin_min, lin_max, n_points_lin, endpoint=False)

    # Logarithmic sampling from lin_max to log_max
    log_samples = np.geomspace(lin_max, log_max, n_points_log)

    # Combine the samples
    combined_samples = np.concatenate([linear_samples, log_samples])

    return combined_samples

## Digitizing data (e.g. plots)

In an ideal world, all scientific data and the models used to describe them would be published alongside the articles that interpret that data. This would be done in a format that allows anyone to replicate the analysis using freely accessible and open-source software (such as pyglotaran), allow for reproducible science.

Fortunately, the practice of publishing data is becoming more common, through increased awareness or mandates from public funding sources. However for many years it was not standard practice to publish data, let alone that there was the infrastructure to do so. As a result, many published results lack accessible datasets. Even if you could contact the authors, they might no longer have the data available.

In these cases, digitizing the printed data and/or results allows us to recreate or simulate the original datasets for re-analysis with modern tools like pyglotaran. This is invaluable for comparing published results to new data, refining experimental parameters, or investigating results when recreating the original experiment is not feasible. 

Thus, we embrace the art of digitization to extend the life and utility of scientific research.

<sub>But before you proceed, do simply try to e-mail one of the original authors, you might get lucky!</sub>

In our example, we will download the SI [PDF](https://pubs.acs.org/doi/suppl/10.1021/jp402086p/suppl_file/jp402086p_si_001.pdf), zoom in as much as possible on the figures of relevant data, and take screenshots. We then load those figures into a digitizer tool (e.g. PlotDigitizer) and start the process of clicking points.

At the end we should end with a list of x,y coordinates for every point we clicked, organized by plot and by trace.

In [ ]:
# Below are the digitized data points from Figure S9 of this paper: https://pubs.acs.org/doi/suppl/10.1021/jp402086p/suppl_file/jp402086p_si_001.pdf
# For this the PlotDigitizer software was used, which can be found here: https://plotdigitizer.com/app

# fmt: off
# ruff: noqa

x_red = [400.7, 409.6, 414.9, 420.9, 429.1, 436.5, 439.7, 453.5, 464.2, 468.4, 470.9, 472.0, 484.0, 498.5, 503.9, 507.4, 508.5, 512.0, 513.8, 515.9, 516.3, 519.4, 520.9, 523.3, 528.0, 535.0, 545.0, 552.1, 555.6, 557.7, 559.1, 561.6, 567.3, 569.1, 570.5, 573.7, 577.2, 580.1, 584.0, 588.9, 591.8, 595.3, 595.7, 600.6, 601.7, 603.4, 604.5, 607.0, 608.1, 610.2, 614.1, 615.1, 617.3, 619.4, 620.8, 622.9, 625.4, 627.6, 629.0, 635.0, 638.9, 643.9, 644.6, 653.1, 659.5, 660.9, 664.1, 673.3, 678.6, 686.0, 690.6, 692.4, 695.6, 698.4, 698.8, 709.4, 723.6, 733.5, 734.9, 740.6, 749.5, 752.3, 752.7, 761.5, 762.2, 772.2]
y_red = [6.6, 4.6, 3.0, 1.3, -2.0, -4.0, -5.0, -6.0, -2.5, -0.7, 0.3, 0.8, 1.9, -0.5, -1.9, -2.3, -3.5, -4.3, -5.6, -6.6, -7.5, -8.1, -10.3, -12.1, -14.7, -17.2, -16.7, -17.3, -16.9, -17.9, -18.6, -21.6, -25.7, -27.3, -28.6, -30.7, -33.5, -37.0, -40.7, -41.7, -43.2, -42.3, -41.6, -39.2, -37.8, -36.7, -35.1, -33.5, -31.8, -28.4, -24.2, -22.6, -20.2, -18.2, -15.9, -14.4, -12.8, -11.1, -9.4, -7.1, -6.6, -5.8, -5.8, -6.1, -6.2, -6.1, -5.3, -3.0, -1.5, 2.0, 3.5, 4.6, 5.9, 6.9, 7.5, 9.0, 9.6, 8.6, 8.4, 8.4, 7.4, 6.4, 6.7, 7.7, 7.2, 8.0]

x_black = [399.65, 401.06, 403.90, 406.03, 417.72, 423.75, 425.17, 425.87, 428.36, 429.77, 431.55, 436.15, 437.93, 440.05, 442.18, 451.39, 457.42, 459.55, 470.18, 473.37, 474.43, 483.29, 484.71, 485.78, 487.55, 492.51, 499.24, 502.08, 503.85, 506.33, 508.46, 513.42, 513.78, 514.13, 519.80, 520.86, 522.64, 526.89, 537.17, 538.94, 541.78, 548.51, 550.64, 552.76, 553.83, 555.60, 556.66, 557.02, 558.44, 558.79, 559.14, 560.92, 562.33, 563.04, 563.75, 564.46, 564.82, 566.23, 566.59, 568.01, 568.36, 569.07, 570.49, 571.20, 571.90, 572.97, 574.03, 575.45, 576.87, 577.58, 578.28, 578.64, 578.64, 579.35, 579.70, 580.06, 581.47, 582.89, 585.37, 586.79, 587.50, 589.27, 589.63, 590.69, 592.46, 592.82, 594.23, 598.84, 602.74, 603.45, 604.51, 605.93, 609.12, 609.83, 610.18, 610.54, 610.89, 613.73, 615.50, 616.56, 617.27, 618.34, 620.11, 620.46, 620.82, 621.53, 622.94, 623.65, 624.72, 625.78, 627.91, 628.62, 631.10, 633.22, 634.64, 635.35, 636.06, 638.19, 640.31, 642.44, 646.34, 649.53, 649.88, 650.59, 651.30, 654.49, 656.26, 657.33, 658.03, 659.81, 661.22, 663.35, 665.12, 666.90, 667.60, 671.15, 672.57, 673.63, 675.05, 677.53, 678.24, 678.59, 679.66, 680.36, 682.14, 683.55, 684.26, 686.74, 689.58, 693.12, 694.54, 695.96, 699.15, 702.69, 703.76, 705.88, 708.36, 709.78, 711.91, 714.04, 716.52, 718.64, 720.77, 723.61, 725.38, 728.57, 734.24, 734.59, 735.30, 739.56, 740.97, 743.81, 744.52, 748.77, 750.19, 758.34, 759.05, 760.11, 760.47, 762.24, 763.66, 767.56, 767.91, 768.27, 768.27]
y_black = [5.14, 5.14, 4.80, 4.69, 2.58, 0.81, 0.03, 0.37, -0.30, -0.85, -1.52, -2.07, -2.74, -3.41, -3.96, -4.74, -4.29, -3.63, -0.63, 0.03, 0.03, 0.48, 0.03, -0.41, -0.19, -0.85, -2.19, -3.07, -3.74, -4.18, -5.29, -6.96, -7.51, -8.18, -12.28, -12.95, -13.50, -14.83, -17.49, -16.83, -17.83, -15.72, -15.83, -17.49, -18.94, -19.93, -21.27, -21.93, -23.71, -24.37, -25.04, -25.93, -26.70, -27.37, -28.03, -29.25, -29.92, -31.80, -32.47, -33.14, -33.80, -35.13, -36.02, -36.69, -37.46, -38.02, -38.46, -39.57, -40.57, -41.23, -42.34, -43.01, -43.67, -44.56, -45.56, -46.23, -46.67, -47.34, -46.89, -47.45, -48.00, -48.11, -48.78, -49.55, -46.89, -46.23, -45.45, -43.34, -39.35, -38.68, -37.35, -35.02, -31.69, -31.03, -30.03, -29.36, -28.70, -25.04, -23.60, -22.26, -21.38, -19.82, -19.71, -19.38, -18.94, -18.27, -17.38, -16.94, -16.27, -15.39, -15.28, -14.83, -14.94, -13.39, -14.39, -13.72, -13.06, -12.72, -11.61, -11.17, -11.17, -11.84, -11.39, -10.84, -10.17, -10.39, -9.95, -9.06, -8.62, -8.40, -8.06, -6.73, -5.96, -5.96, -5.29, -2.85, -1.74, -0.85, -0.08, 0.26, 0.92, 1.48, 1.92, 2.25, 2.81, 4.03, 4.69, 5.91, 7.47, 7.80, 7.91, 8.69, 8.91, 10.68, 11.79, 11.79, 11.24, 12.01, 12.24, 11.79, 12.46, 12.57, 12.68, 12.35, 12.13, 11.79, 11.68, 11.02, 10.35, 11.13, 9.57, 10.13, 10.68, 9.13, 8.13, 8.46, 9.13, 8.69, 8.02, 8.46, 8.24, 8.58, 8.13, 6.47, 5.80]

x_green = [401.77, 404.61, 406.03, 409.92, 410.63, 413.47, 416.66, 419.85, 421.62, 426.23, 431.19, 434.74, 439.34, 444.31, 451.04, 454.23, 458.13, 459.90, 463.09, 466.28, 471.24, 474.79, 475.85, 480.81, 485.78, 489.32, 493.57, 496.41, 499.60, 502.79, 506.33, 509.52, 512.00, 514.13, 517.67, 518.74, 523.35, 526.18, 530.43, 538.94, 541.78, 546.38, 549.22, 553.12, 554.18, 554.89, 558.79, 560.56, 565.88, 570.13, 575.09, 578.99, 582.18, 584.31, 585.73, 588.21, 589.98, 592.82, 596.72, 598.49, 600.61, 604.51, 611.25, 614.08, 616.92, 620.82, 622.24, 625.78, 627.91, 631.81, 632.87, 637.12, 641.38, 652.36, 656.62, 656.62, 659.45, 664.06, 665.83, 666.54, 667.60, 668.67, 670.79, 672.92, 675.05, 678.95, 683.55, 685.68, 688.52, 690.64, 691.35, 692.41, 695.25, 699.50, 702.34, 703.05, 703.76, 705.53, 711.91, 713.68, 715.45, 716.52, 719.71, 720.77, 721.83, 722.90, 725.38, 728.21, 731.40, 731.76, 733.18, 738.85, 739.56, 740.97, 741.33, 742.39, 743.81, 744.16, 744.52, 745.23, 745.94, 748.06, 748.77, 749.83, 752.32, 753.02, 753.38, 753.73, 754.09, 755.15, 755.51, 756.57, 756.92, 757.63, 758.34, 758.70, 759.05, 759.40, 759.76, 760.47, 763.66, 766.49, 766.85, 770.04, 770.39, 770.75, 771.10, 771.46, 771.81, 772.16, 772.52, 773.23, 773.58, 774.65]
y_green = [1.81, 1.36, 0.81, 0.70, -0.30, -0.85, -0.85, -1.63, -2.52, -3.52, -4.40, -5.62, -5.40, -5.85, -5.40, -4.51, -3.63, -2.19, -0.85, 0.48, 1.92, 3.14, 1.92, 2.25, 1.48, 0.92, -0.52, -1.85, -3.18, -4.18, -5.18, -6.07, -7.51, -9.29, -9.95, -11.39, -12.61, -13.72, -14.50, -16.27, -17.27, -17.16, -16.50, -16.72, -17.83, -18.38, -19.38, -19.82, -19.71, -19.38, -19.49, -18.71, -16.27, -16.16, -14.39, -14.50, -13.72, -12.06, -11.06, -10.17, -8.73, -6.62, -5.51, -3.74, -3.18, -2.63, -1.19, -1.08, -0.52, 0.70, 2.58, 1.59, 2.36, 4.25, 3.36, 3.25, 4.91, 5.36, 3.92, 2.69, 4.91, 3.47, 4.69, 4.03, 5.03, 4.80, 5.25, 5.58, 7.69, 6.25, 6.80, 8.24, 9.46, 9.68, 9.46, 8.02, 9.02, 10.46, 9.13, 7.58, 8.91, 9.46, 10.13, 8.58, 9.24, 10.68, 10.91, 11.35, 12.79, 12.35, 11.90, 12.57, 13.46, 11.46, 12.24, 14.57, 12.35, 12.79, 14.23, 12.35, 10.79, 10.24, 12.35, 10.91, 9.80, 10.13, 11.57, 12.79, 14.23, 11.46, 9.57, 7.69, 8.24, 9.68, 11.68, 12.46, 13.90, 15.34, 16.23, 14.68, 14.57, 12.46, 13.01, 14.57, 15.01, 16.45, 17.89, 19.34, 20.78, 22.11, 23.55, 22.22, 19.45, 17.45]

x_cyan = [419.49, 421.98, 425.17, 425.87, 428.00, 428.71, 430.13, 431.19, 434.38, 437.22, 440.76, 443.24, 446.79, 448.20, 448.20, 449.27, 449.98, 452.46, 454.58, 456.00, 456.71, 458.13, 460.96, 462.38, 465.57, 468.76, 471.24, 473.37, 474.43, 476.56, 478.69, 479.40, 480.46, 485.78, 488.61, 490.38, 492.16, 493.22, 493.57, 496.41, 500.31, 502.08, 504.21, 504.91, 507.04, 507.75, 508.46, 508.81, 512.00, 513.07, 514.13, 515.19, 516.61, 518.38, 519.09, 520.16, 521.57, 522.99, 525.47, 527.60, 528.66, 530.08, 530.43, 531.14, 533.27, 536.46, 536.81, 538.94, 541.42, 542.84, 544.26, 547.09, 548.16, 549.22, 550.28, 552.06, 555.25, 555.95, 556.31, 558.44, 559.14, 559.50, 560.92, 561.27, 561.98, 565.17, 566.94, 567.30, 568.71, 570.49, 571.20, 572.26, 572.26, 572.61, 573.68, 573.68, 574.74, 575.80, 576.51, 576.87, 577.58, 580.41, 582.18, 582.54, 583.60, 585.02, 585.73, 587.85, 589.27, 591.04, 591.75, 592.46, 593.17, 593.53, 594.23, 594.59, 595.30, 596.01, 596.36, 596.72, 597.78, 599.20, 599.91, 600.97, 601.68, 602.39, 603.80, 604.87, 606.99, 608.06, 609.48, 611.60, 612.31, 613.37, 615.86, 619.05, 621.17, 624.72, 625.78, 628.62, 631.10, 631.45, 633.22, 634.64, 636.77, 640.67, 643.50, 644.92, 645.98, 649.53, 653.07, 655.91, 658.39, 660.16]
y_cyan = [0.59, -1.19, -1.08, -2.19, -2.74, -3.29, -2.19, -1.85, -3.96, -3.41, -4.51, -4.51, -6.62, -3.96, -2.52, -4.63, -5.96, -6.73, -5.85, -7.51, -6.29, -5.29, -6.07, -4.63, -3.41, -2.07, -0.63, 0.37, 1.70, -0.30, 1.70, 1.81, 1.03, 2.03, 1.92, 3.36, 3.58, 1.81, 2.36, 3.36, 2.47, 1.36, 2.58, 2.58, 3.14, 1.70, 0.26, -0.52, 1.36, -0.74, -2.19, -3.63, -5.07, -4.74, -4.07, -5.51, -6.96, -8.40, -9.29, -9.95, -11.28, -8.84, -10.28, -11.17, -9.84, -9.51, -8.06, -7.07, -6.29, -4.85, -3.41, -2.19, -2.52, -3.96, -5.40, -6.40, -6.62, -8.06, -9.51, -10.84, -12.28, -13.72, -15.16, -16.61, -18.05, -20.49, -21.93, -23.37, -24.82, -26.26, -27.70, -29.14, -30.92, -29.92, -29.03, -27.70, -29.92, -28.70, -26.04, -27.26, -29.03, -30.25, -31.36, -30.36, -28.92, -27.48, -26.04, -24.71, -23.26, -22.04, -20.60, -19.16, -17.72, -16.27, -14.83, -13.39, -11.95, -10.51, -9.06, -7.62, -6.18, -4.74, -3.29, -1.85, -0.41, 1.14, 2.58, 4.03, 5.47, 6.91, 8.35, 9.46, 11.24, 9.80, 9.91, 10.35, 9.02, 9.02, 7.58, 7.80, 7.47, 6.02, 4.58, 6.25, 4.80, 4.91, 3.69, 2.25, 3.36, 3.14, 2.47, 1.25, 1.48, 0.26]
# fmt: on

In [ ]:
# As part of the manual labeling process and the finite precision of the digitization process, the data points are not perfectly aligned,
# and sometimes duplicate (x) values are present, thus we remove those first.
spec_red = convert_to_xarray(*(interpolate_with_xarray(*remove_duplicates(x_red, y_red))))
spec_black = convert_to_xarray(*(interpolate_with_xarray(*remove_duplicates(x_black, y_black))))
spec_green = convert_to_xarray(*(interpolate_with_xarray(*remove_duplicates(x_green, y_green))))
spec_cyan = convert_to_xarray(*(interpolate_with_xarray(*remove_duplicates(x_cyan, y_cyan))))

## Plot cleaned up spectra


In [ ]:
# plot the spectra
from matplotlib import pyplot as plt

fig, ax = plt.subplots()
spec_red.plot(ax=ax, label="Red", color="red")
spec_black.plot(ax=ax, label="Black", color="black")
spec_green.plot(ax=ax, label="Green", color="green")
spec_cyan.plot(ax=ax, label="Cyan", color="cyan")
ax.set_xlabel("Wavelength [nm]")
ax.set_ylabel("Intensity [mOD]")
ax.legend()  # Add legend to the plot

Compare this to the original figure (linked below)

<a href="https://pubs.acs.org/doi/suppl/10.1021/jp402086p/suppl_file/jp402086p_si_001.pdf#page=20">Open SI on page 20</a>


## Extract kinetics

In addition to the spectra we need to extract the kinetic scheme and the reported rate constant. Fortuoantely for us, the target scheme and most of the relevant parameters van be directly extracted from Figure S8 on page 19.

<a href="https://pubs.acs.org/doi/suppl/10.1021/jp402086p/suppl_file/jp402086p_si_001.pdf#page=19">Open SI on page 19</a>


### Extracting the target scheme and rate constants

The target scheme in the SI figure (redrawn below) allows us to fully deduce the scheme.

![target scheme](figures/Figure_S8_Target_Scheme-dark.drawio.svg)

Based on the population profiles from Figure S9 in the SI we know that 100% of the population goes into the first excited state (FC), or compartment A.

### Translating to pyglotaran model syntax

And putting them in the model (matrix of rate constants).

```yaml
k_matrix:
  km1:
    matrix:                                # rates ps^-1
      (B, A): "rates.k1"     # FC  -> *S1  # 1.53 
      (C, B): "rates.k2"     # S1* -> S1   # 0.103
      (D, C): "rates.k3"     # S1  -> EX   # ~0
      (E, D): "rates.k4"     # EX  -> T    # 0.076/1000
      (GS, A): "rates.k5"    # A   -> GS   # 0.60
      (GS, B): "rates.k6"    # B   -> GS   # 0.19
      (GS, C): "rates.k7"    # C   -> GS   # 0.034
      (GS, D): "rates.k8"    # D   -> GS   # 5.14/1000
      (GS, E): "rates.k9"    # A   -> GS   # 0.18/1000
      (D, B): "rates.k10"    # *S1 -> EX   # 0.07
      (E, C): "rates.k11"    # S1  -> T    # 22.6/1000
```

The rates

```yaml
rates:
  - ['k1', 1.53]
  - ['k2', 0.103]
  - ['k3', 1.0e-8]
  - ['k4', 7.6e-5]
  - ['k5', 0.60]
  - ['k6', 0.19]
  - ['k7', 0.034]
  - ['k8', 0.00514]
  - ['k9', 0.00018]
  - ['k10', 0.07]
  - ['k11', 0.0226]
  ```

  The inputs
  
  ```yaml
  inputs:
  - ['A', 1]
  - ['the_rest', 0]
  ```

In [ ]:
## Extract other relevant
[0.076 / 1000, 5.14 / 1000, 0.18 / 1000, 22.6 / 1000]
# upper bound for C->D rate, which I will just assume to be close to 0 for now
1 / 17.5 - (0.034 + 22.6 / 1000)

## Extract other relevant information

In addition to the kinetic scheme, the associated spectral amplitudes, we need some more information to accurately simulate the data. 

- The instrument response function's (IRF) characteristics
- The time and spectral window and approximate sampling
- (optional) noise charactecteristis


### Instrument reponse function

The IRF is typically described as some kind of gaussian-like function with a center and a width sigma or full with half maximum (FWHM). Ideally you can obtian this directly from the text in the paper, otherwise you make a guestimate based on a (fitted) time trace in the paper and then refine this in simulation later.

In this case we can guestimate the IRF's FWHM to be about 0.05ps.

### Simulation window

For a representative simulation we should roughly match the sampling density and the time and spectral windows of the original data. 

Based on the descriptions from the text and the figures we can deduce that there are 256 wavelength samples between 400 and 775 nm, and in Figure S10 75 fitted traces are plotted every 4-5 nm between 401 nm annd 713 nm, which can later be used for validation of our simulation.

The time range run from just before time zero (-1ps) to 4000 ps, but the time axis is plotted linear from -4 to 4 ps, and logarithmic from 4 to 4000ps. 

### Noise characteristics

For a more realistic simulation it often helps to add (realistic) noise. From the fitted traces at low amplitudes (e.g. in the window 430-440nm) we can guestimate that the standard deviation in the noise is around 0.5 units which we presume are mOD.

## Putting it all together

We're going to 
- Load in the spectral shapes
- Define the model and (simulation) parameters
- Simulate the data
- Plot the data
- Fit the data (with the same model)
- Fit the data (with another model)
- Plot the fits

### Define spectra

In [ ]:
# ruff: noqa
# fmt: off

x_A = [400.00000, 403.89887, 409.92439, 411.34216, 414.88658, 417.72212, 420.91210, 424.81096, 426.93762, 429.41871, 432.60870, 436.50756, 440.05198, 445.72306, 450.33081, 451.74858, 458.12854, 459.90076, 461.67297, 465.57183, 470.17958, 472.66068, 476.55955, 480.45841, 483.29395, 484.35728, 487.54726, 491.09168, 493.21834, 495.69943, 498.53497, 499.59830, 500.66163, 502.07940, 503.85161, 507.04159, 508.45936, 510.94045, 513.06711, 514.13043, 519.80151, 520.86484, 524.76371, 529.72590, 532.56144, 533.97921, 536.81474, 537.87807, 540.35917, 541.42250, 545.32136, 546.38469, 546.38469, 550.28355, 552.05577, 552.76465, 553.82798, 555.24575, 556.30907, 557.37240, 558.08129, 558.79017, 560.91682, 563.04348, 563.75236, 564.81569, 565.52457, 566.23346, 568.00567, 568.71456, 569.77788, 571.55009, 572.25898, 574.03119, 575.09452, 576.86673, 577.93006, 578.63894, 578.99338, 579.70227, 580.76560, 583.60113, 584.66446, 587.50000, 588.56333, 589.98110, 590.68998, 592.10775, 592.46219, 593.17108, 596.71550, 597.06994, 598.84216, 599.90548, 600.61437, 602.03214, 603.80435, 604.86767, 605.57656, 608.76654, 610.18431, 610.53875, 610.89319, 614.08318, 615.14650, 616.56427, 617.27316, 617.98204, 619.75425, 620.81758, 622.23535, 624.36200, 627.19754, 629.32420, 631.09641, 631.09641, 631.45085, 633.22306, 634.64083, 635.70416, 636.41304, 637.83081, 638.53970, 640.31191, 642.43856, 644.21078, 646.33743, 648.81853, 650.23629, 651.65406, 654.13516, 655.55293, 657.32514, 658.38847, 661.22401, 662.64178, 665.12287, 666.54064, 668.66730, 670.79395, 672.56616, 673.98393, 677.88280, 678.59168, 681.42722, 683.55388, 686.74386, 688.51607, 689.57940, 692.41493, 693.47826, 695.25047, 696.31380, 697.73157, 700.92155, 702.69376, 704.46597, 706.94707, 708.36484, 709.42817, 711.55482, 713.32703, 715.45369, 717.22590, 719.70699, 721.12476, 722.18809, 725.02363, 727.50473, 729.63138, 731.40359, 732.82136, 734.23913, 735.30246, 737.07467, 738.49244, 739.55577, 740.97353, 741.68242, 742.39130, 743.80907, 744.87240, 748.77127, 750.18904, 751.60681, 753.37902, 754.08790, 755.86011, 757.63233, 759.75898, 760.46786, 762.94896, 764.01229, 764.72117, 766.84783, 767.55671, 767.91115, 768.26560, 768.26560, 769.68336, 770.03781, 772.87335, 773.58223, 773.93667]
y_A = [5.00000, 4.77951, 4.44444, 3.55556, 2.77951, 2.66840, 1.66840, 0.55729, 0.11111, -0.66667, -1.55382, -2.22049, -3.44271, -4.55382, -4.33333, -4.99826, -4.33160, -3.44444, -3.10938, -1.44271, -0.66493, 0.00174, 0.00174, 0.44618, 0.55556, -0.10938, -0.33333, -0.44444, -1.22222, -1.77778, -1.77778, -2.44271, -3.11111, -3.00000, -3.77604, -4.33160, -5.55382, -6.88715, -6.88889, -8.10938, -11.99826, -13.22049, -14.33160, -15.44271, -16.77604, -17.33160, -17.55382, -16.99826, -17.10938, -17.88715, -17.44271, -16.55382, -16.99826, -15.55382, -16.88715, -17.44271, -18.66493, -19.88715, -21.10938, -22.33160, -23.55382, -24.77604, -25.99826, -27.22049, -28.44271, -29.66493, -30.88715, -32.10938, -33.33160, -34.55382, -35.77604, -36.99826, -38.22049, -38.22049, -39.44271, -40.66493, -41.88715, -43.10938, -44.33160, -45.55382, -46.77604, -47.99826, -47.10938, -48.33160, -48.10938, -49.33160, -50.00000, -48.10938, -46.88715, -45.66493, -44.44271, -43.22049, -43.22049, -41.99826, -40.77604, -39.55382, -38.33160, -37.10938, -35.88715, -31.99826, -30.77604, -29.55382, -28.33160, -25.10938, -23.88715, -22.66493, -21.44271, -20.22049, -20.22049, -18.99826, -17.77604, -16.55382, -15.33160, -14.66667, -14.66493, -15.44444, -13.33333, -13.44271, -14.66493, -13.44271, -12.88889, -13.22222, -12.22049, -11.66667, -11.22049, -12.00000, -11.22049, -12.22222, -11.33160, -10.10938, -10.44271, -10.66667, -9.22049, -8.44444, -8.22049, -6.99826, -6.00000, -6.33160, -4.22049, -2.99826, -1.77604, -0.55382, 0.22396, 1.44618, 2.66840, 3.89063, 5.77951, 7.00174, 7.55729, 6.66840, 8.11111, 7.89063, 9.11285, 8.89063, 10.00174, 10.77951, 12.00174, 12.11285, 10.89063, 12.00174, 12.33333, 11.77951, 11.66667, 12.66840, 12.22222, 12.66840, 12.00000, 12.33507, 12.22396, 11.22396, 11.77951, 12.44618, 11.77951, 10.22222, 10.44444, 10.22222, 11.33333, 9.44444, 10.22396, 10.77778, 9.88889, 10.66667, 9.11111, 8.00000, 8.44618, 8.33507, 7.77778, 8.55556, 8.00000, 9.55556, 8.00174, 8.55556, 8.00174, 7.22396, 8.22222, 8.55556, 7.55729, 6.55729, 5.33507, 6.44444, 7.44618, 8.66667, 7.11285, 6.33507]

x_BC = [400.70888, 409.56994, 414.88658, 420.91210, 429.06427, 436.50756, 439.69754, 453.52079, 464.15406, 468.40737, 470.88847, 471.95180, 484.00284, 498.53497, 503.85161, 507.39603, 508.45936, 512.00378, 513.77599, 515.90265, 516.25709, 519.44707, 520.86484, 523.34594, 527.95369, 535.04253, 544.96692, 552.05577, 555.60019, 557.72684, 559.14461, 561.62571, 567.29679, 569.06900, 570.48677, 573.67675, 577.22117, 580.05671, 583.95558, 588.91777, 591.75331, 595.29773, 595.65217, 600.61437, 601.67769, 603.44991, 604.51323, 606.99433, 608.05766, 610.18431, 614.08318, 615.14650, 617.27316, 619.39981, 620.81758, 622.94423, 625.42533, 627.55198, 628.96975, 634.99527, 638.89414, 643.85633, 644.56522, 653.07183, 659.45180, 660.86957, 664.05955, 673.27505, 678.59168, 686.03497, 690.64272, 692.41493, 695.60491, 698.44045, 698.79490, 709.42817, 723.60586, 733.53025, 734.94802, 740.61909, 749.48015, 752.31569, 752.67013, 761.53119, 762.24008, 772.16446]
y_BC = [6.57686, 4.58003, 3.02694, 1.25371, -1.96513, -3.96197, -4.96038, -5.95706, -2.51808, -0.74312, 0.25530, 0.80997, 1.91759, -0.52125, -1.85247, -2.29794, -3.51823, -4.29304, -5.62426, -6.62267, -7.51015, -8.06483, -10.28353, -12.05849, -14.72093, -17.16323, -16.71776, -17.27243, -16.93963, -17.93804, -18.60365, -21.59890, -25.70350, -27.25659, -28.58781, -30.69557, -33.46895, -37.02060, -40.67972, -41.67987, -43.23296, -42.34375, -41.56720, -39.23757, -37.79541, -36.68606, -35.13297, -33.47068, -31.80492, -28.36767, -24.15041, -22.59732, -20.15675, -18.16165, -15.94121, -14.38986, -12.83503, -11.06007, -9.39605, -7.06815, -6.62441, -5.84613, -5.84613, -6.06973, -6.17893, -6.06800, -5.29145, -2.96182, -1.51966, 2.03026, 3.47241, 4.58177, 5.91299, 6.91140, 7.46608, 9.01917, 9.57384, 8.57543, 8.35356, 8.35182, 7.35514, 6.35673, 6.68953, 7.68795, 7.24421, 8.02075]

x_D = [401.06333, 402.83554, 405.67108, 406.73440, 410.63327, 411.34216, 412.75992, 415.24102, 417.36767, 419.13989, 422.32987, 426.22873, 430.12760, 431.19093, 434.02647, 436.86200, 440.76087, 445.36862, 447.14083, 448.20416, 450.33081, 453.16635, 453.87524, 457.77410, 459.54631, 462.73629, 465.92628, 468.76181, 471.59735, 475.49622, 477.97732, 481.16730, 482.23062, 486.12949, 490.02836, 493.92722, 496.05388, 499.24386, 501.37051, 505.26938, 508.81380, 510.58601, 513.06711, 514.48488, 516.61153, 517.32042, 518.73819, 522.63705, 524.40926, 526.18147, 530.08034, 533.97921, 537.87807, 541.77694, 545.67580, 549.57467, 553.47353, 553.82798, 557.72684, 561.62571, 562.68904, 564.81569, 566.58790, 568.71456, 572.61342, 576.51229, 579.34783, 580.76560, 582.18336, 583.60113, 584.66446, 585.37335, 586.08223, 589.27221, 590.33554, 591.75331, 596.36106, 596.71550, 597.06994, 599.55104, 600.96881, 602.74102, 605.57656, 609.47543, 612.31096, 613.37429, 615.50095, 618.69093, 619.39981, 621.17202, 622.94423, 625.07089, 626.13422, 626.84310, 629.32420, 631.80529, 632.15974, 633.57750, 635.34972, 637.12193, 641.02079, 645.98299, 647.04631, 648.81853, 649.17297, 653.07183, 657.32514, 660.51512, 664.41399, 666.89509, 667.60397, 669.02174, 670.43951, 671.85728, 674.69282, 675.75614, 679.65501, 683.55388, 685.68053, 686.38941, 687.45274, 688.16163, 689.57940, 690.99716, 691.35161, 692.41493, 694.18715, 696.31380, 697.37713, 699.14934, 701.98488, 703.04820, 705.88374, 707.30151, 708.36484, 709.78261, 711.55482, 712.26371, 714.03592, 716.16257, 719.35255, 720.41588, 721.12476, 721.83365, 722.89698, 724.66919, 726.79584, 728.92250, 731.40359, 733.17580, 737.07467, 739.55577, 740.97353, 740.97353, 742.39130, 744.51796, 744.87240, 745.58129, 748.06238, 749.12571, 751.96125, 753.02457, 753.37902, 753.37902, 754.08790, 755.15123, 755.15123, 755.50567, 755.86011, 756.56900, 757.98677, 758.34121, 758.69565, 759.05009, 759.40454, 759.75898, 764.01229, 766.49338, 767.20227, 770.39225, 770.74669, 771.10113, 771.45558, 771.45558, 772.51890, 772.87335, 773.58223, 773.58223, 773.93667, 774.29112]
y_D = [2.33507, 1.11285, 1.77778, 0.33507, 0.22396, -1.00000, 0.22222, -1.55382, -0.44444, -1.44271, -2.66493, -3.55382, -4.44271, -4.22049, -5.44271, -5.44271, -5.88715, -5.88715, -5.99826, -5.99826, -5.55382, -5.44271, -4.55382, -3.77604, -2.22049, -0.99826, 0.22396, 1.44618, 2.22396, 2.55729, 2.55729, 2.22396, 2.11285, 1.33507, 0.89063, -0.55382, -1.77604, -2.99826, -4.22049, -4.66493, -5.88715, -7.10938, -8.33160, -9.77778, -9.55382, -10.10938, -11.33160, -12.55382, -13.77604, -13.77604, -14.44271, -15.55382, -16.33160, -17.33160, -17.22049, -16.66493, -16.66493, -17.88715, -19.10938, -19.55382, -19.33160, -19.33160, -20.00000, -19.22049, -19.44271, -19.66493, -18.44271, -17.22049, -16.33333, -17.11111, -16.22049, -14.99826, -13.77604, -14.55382, -13.33160, -12.10938, -12.10938, -10.88715, -9.66493, -9.66493, -8.44271, -7.22049, -5.99826, -5.10938, -5.77778, -4.77604, -2.77604, -3.88889, -3.33160, -2.10938, -0.88715, -0.66667, -1.77778, -1.22049, -0.55556, 0.44444, 1.33507, 2.55729, 1.33507, 1.55729, 2.22396, 3.00174, 3.11285, 2.77951, 3.44618, 4.22396, 3.66840, 4.89063, 5.22396, 2.77778, 5.22222, 3.66667, 5.22396, 4.00174, 3.88889, 5.22396, 4.77951, 5.22396, 4.66667, 6.44618, 7.66840, 8.22222, 6.66667, 6.00000, 6.89063, 8.11285, 9.11285, 9.44444, 9.00000, 10.00000, 9.66840, 8.44618, 10.66840, 11.22222, 10.00000, 11.11285, 9.89063, 8.66840, 7.44444, 9.22396, 10.44618, 9.22396, 8.22222, 9.44618, 10.66840, 10.55556, 11.66840, 11.11285, 13.11285, 11.89063, 12.11285, 13.22396, 12.00174, 10.77951, 14.00174, 14.00174, 12.77951, 11.55729, 10.33507, 11.44618, 10.22396, 9.00174, 10.22396, 11.44618, 14.33333, 12.22396, 11.00174, 9.77951, 8.55729, 7.33507, 9.77951, 11.00174, 12.22396, 13.44618, 14.66840, 15.89063, 14.11285, 12.89063, 14.11285, 15.33507, 16.55729, 17.77951, 20.22396, 19.00174, 22.66840, 23.89063, 20.22396, 21.44618, 19.00174, 17.77951]

x_E = [419.49433, 421.97543, 425.16541, 425.87429, 428.00095, 428.70983, 430.12760, 431.19093, 434.38091, 437.21645, 440.76087, 443.24197, 446.78639, 448.20416, 448.20416, 449.26749, 449.97637, 452.45747, 454.58412, 456.00189, 456.71078, 458.12854, 460.96408, 462.38185, 465.57183, 468.76181, 471.24291, 473.36957, 474.43289, 476.55955, 478.68620, 479.39509, 480.45841, 485.77505, 488.61059, 490.38280, 492.15501, 493.21834, 493.57278, 496.40832, 500.30718, 502.07940, 504.20605, 504.91493, 507.04159, 507.75047, 508.45936, 508.81380, 512.00378, 513.06711, 514.13043, 515.19376, 516.61153, 518.38374, 519.09263, 520.15595, 521.57372, 522.99149, 525.47259, 527.59924, 528.66257, 530.08034, 530.43478, 531.14367, 533.27032, 536.46030, 536.81474, 538.94140, 541.42250, 542.84026, 544.25803, 547.09357, 548.15690, 549.22023, 550.28355, 552.05577, 555.24575, 555.95463, 556.30907, 558.43573, 559.14461, 559.49905, 560.91682, 561.27127, 561.98015, 565.17013, 566.94234, 567.29679, 568.71456, 570.48677, 571.19565, 572.25898, 572.25898, 572.61342, 573.67675, 573.67675, 574.74008, 575.80340, 576.51229, 576.86673, 577.57561, 580.41115, 582.18336, 582.53781, 583.60113, 585.01890, 585.72779, 587.85444, 589.27221, 591.04442, 591.75331, 592.46219, 593.17108, 593.52552, 594.23440, 594.58885, 595.29773, 596.00662, 596.36106, 596.71550, 597.77883, 599.19660, 599.90548, 600.96881, 601.67769, 602.38658, 603.80435, 604.86767, 606.99433, 608.05766, 609.47543, 611.60208, 612.31096, 613.37429, 615.85539, 619.04537, 621.17202, 624.71645, 625.77977, 628.61531, 631.09641, 631.45085, 633.22306, 634.64083, 636.76749, 640.66635, 643.50189, 644.91966, 645.98299, 649.52741, 653.07183, 655.90737, 658.38847, 660.16068]
y_E = [0.58810, -1.18686, -1.07592, -2.18527, -2.73995, -3.29462, -2.18527, -1.85247, -3.96023, -3.40556, -4.51491, -4.51491, -6.62267, -3.96023, -2.51808, -4.62584, -5.95706, -6.73361, -5.84613, -7.51015, -6.28987, -5.29145, -6.06800, -4.62584, -3.40556, -2.07434, -0.63218, 0.36623, 1.69745, -0.29938, 1.69745, 1.80839, 1.03184, 2.03026, 1.91932, 3.36148, 3.58335, 1.80839, 2.36306, 3.36148, 2.47227, 1.36292, 2.58493, 2.58493, 3.13961, 1.69745, 0.25530, -0.52125, 1.36465, -0.74312, -2.18527, -3.62743, -5.06958, -4.73678, -4.07117, -5.51332, -6.95548, -8.39763, -9.28511, -9.95072, -11.28194, -8.84137, -10.28353, -11.17101, -9.83979, -9.50698, -8.06483, -7.06641, -6.28987, -4.84771, -3.40556, -2.18527, -2.51808, -3.96023, -5.40239, -6.40080, -6.62267, -8.06483, -9.50698, -10.83820, -12.28036, -13.72251, -15.16467, -16.60682, -18.04898, -20.48955, -21.93171, -23.37386, -24.81602, -26.25817, -27.70033, -29.14248, -30.91918, -29.91903, -29.03155, -27.70033, -29.91903, -28.69874, -26.03803, -27.25659, -29.03155, -30.25183, -31.36292, -30.36277, -28.92061, -27.47846, -26.03630, -24.70508, -23.26293, -22.04264, -20.60049, -19.15833, -17.71617, -16.27402, -14.83186, -13.38971, -11.94755, -10.50540, -9.06324, -7.62109, -6.17893, -4.73678, -3.29462, -1.85247, -0.41031, 1.14278, 2.58493, 4.02709, 5.46925, 6.91140, 8.35356, 9.46291, 11.23787, 9.79571, 9.90665, 10.35039, 9.01917, 9.01917, 7.57701, 7.79888, 7.46608, 6.02392, 4.58177, 6.24579, 4.80364, 4.91457, 3.69428, 2.25213, 3.36148, 3.13961, 2.47400, 1.25371, 1.47558, 0.25530]

# fmt: on
# As part of the manual labeling process and the finite precision of the digitization process, the data points are not perfectly aligned,
# and sometimes duplicate (x) values are present, thus we remove those first.
spectrum_A = convert_to_xarray(
    *(interpolate_with_xarray(*remove_duplicates(x_A, y_A), xrange=(400, 775), num_points=256))
)
spectrum_BC = convert_to_xarray(
    *(interpolate_with_xarray(*remove_duplicates(x_BC, y_BC), xrange=(400, 775), num_points=256))
)
spectrum_D = convert_to_xarray(
    *(interpolate_with_xarray(*remove_duplicates(x_D, y_D), xrange=(400, 775), num_points=256))
)
spectrum_E = convert_to_xarray(
    *(interpolate_with_xarray(*remove_duplicates(x_E, y_E), xrange=(400, 775), num_points=256))
)

assert 256 == len(spectrum_A) == len(spectrum_BC) == len(spectrum_D) == len(spectrum_E)

In [ ]:
# List of DataArrays
spectrum_list = [spectrum_A, spectrum_BC, spectrum_BC, spectrum_D, spectrum_E]
# List of custom clp_labels
clp_labels = ["A", "B", "C", "D", "E"]
# Combine them along the 'clp_label' axis with custom labels
clps = xr.concat(spectrum_list, dim="clp_label").fillna(0)
clps = clps.assign_coords(clp_label=clp_labels)

temporal_axis = linear_log_sampling(-4, 4, 4000, 50, 50)  # timepoints in ps
spectral_axis = clps.coords["spectral"].to_numpy()  # wavelength in nm

simulation_coordinates = {"time": temporal_axis, "spectral": spectral_axis}

In [ ]:
from glotaran.io import load_parameters
from glotaran.io import load_model

simulation_model = load_model("models/simulation_model.yml", format_name="yml")
simulation_parameters = load_parameters("models/simulation_parameters.yml", format_name="yml")

In [ ]:
from glotaran.simulation import simulate

dataset1 = simulate(
    simulation_model,
    "dataset1",
    simulation_parameters,
    simulation_coordinates,
    clp=clps,
    noise=True,
    noise_std_dev=0.75,
)

In [ ]:
from pyglotaran_extras.plotting import plot_data_overview

plot_data_overview(dataset1, linlog=True);

## Fit the data

In [ ]:
from glotaran.project.scheme import Scheme

target_scheme = Scheme(
    model=load_model("models/target_model.yml"),
    parameters=load_parameters("models/target_parameters.yml"),
    maximum_number_function_evaluations=99,
    data={
        "dataset1": dataset1,
    },
)
target_scheme.validate()

In [ ]:
from glotaran.optimization.optimize import optimize

target_result = optimize(target_scheme, raise_exception=True)

# Plotting

Time to make the magic happen!

In [ ]:
# This allows us to use a config file to customize plots
from pyglotaran_extras import CONFIG

CONFIG.init_project()  # if already initialized, this does nothing

In [ ]:
from cycler import cycler
from pyglotaran_extras.plotting.style import PlotStyle
from pyglotaran_extras import plot_overview
from pyglotaran_extras.plotting.style import ColorCode

custom_si_cycler = cycler(
    color=[ColorCode.black, ColorCode.blue, ColorCode.red, ColorCode.green4, ColorCode.cyan]
)

plot_overview(
    target_result,
    linthresh=4,
    cycler=custom_si_cycler,
    das_cycler=PlotStyle().cycler,
    svd_cycler=PlotStyle().cycler,
);

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths


axis_shape = (15, 5)
wavelengths = select_plot_wavelengths(
    target_result, axes_shape=axis_shape, wavelength_range=(405.5, 713.4)
)
wavelengths = np.flip(wavelengths.reshape((5, 15)).T, axis=0).flatten()
plot_fitted_traces(
    target_result,
    linlog=True,
    linthresh=4,
    axes_shape=axis_shape,
    wavelengths=wavelengths,
    figsize=(15, 2 * 15),
    cycler=PlotStyle().cycler,
);

Compare this to the original figure (linked below)

<a href="https://pubs.acs.org/doi/suppl/10.1021/jp402086p/suppl_file/jp402086p_si_001.pdf#page=22">Open SI on page 22</a>